In [ ]:
import matplotlib.pyplot as plt
from astropy.io import fits
import numpy as np
from matplotlib.patches import Rectangle
from astropy.visualization import simple_norm
from scipy.ndimage import gaussian_filter
from matplotlib.colors import Normalize
from matplotlib import gridspec
import os
from matplotlib.colors import LogNorm
from matplotlib.ticker import LogLocator, LogFormatterMathtext
from matplotlib.ticker import LogLocator, LogFormatterMathtext

In [ ]:
# get pixle resolution
import soxs

instrument = soxs.instrument_registry["chandra_acisi_cy22"] 
fov_arcmin = instrument["fov"]          
num_pixels = instrument["num_pixels"] 

pixel_size_arcsec = (fov_arcmin * 60) / num_pixels
print("Pixel size (arcsec):", pixel_size_arcsec)



In [ ]:
pixel_size_arcsec

In [ ]:
from astropy.cosmology import Planck15 as cosmo
import astropy.units as u

z = 0.2
d_A = cosmo.angular_diameter_distance(z)
theta_rad = (1 / 3600) * (np.pi / 180) # 1 arcsec 
length_per_arcsec = (d_A * theta_rad).to(u.kpc)

length_kpc = 1000
length_arcsec = (length_kpc * u.kpc) / length_per_arcsec

length_pix = length_arcsec.value / pixel_size_arcsec


In [ ]:
import pandas as pd

csv_path = "/users_path/merger_trace/data/tng_cluster/tng_cluster_track2snapx/fof_halo_to_sub84.csv"
df = pd.read_csv(csv_path)
r_crit500 = df['Group_R_Crit500'][0] # ckpc/h

In [ ]:
h = cosmo.h

r_crit500_kpc = r_crit500/h * 1/(1+z)

r_arcsec = (r_crit500_kpc* u.kpc) / length_per_arcsec

r_pix = r_arcsec/pixel_size_arcsec

In [ ]:
base_path = '/users_path/merger_trace/data/tng_cluster/tng_cluster_Xray_img/'
x_path = base_path + "halo_0_img_x.fits"
y_path = base_path + "halo_0_img_y.fits"
z_path = base_path + "halo_0_img_z.fits"
x_path_new = base_path + "halo_0_img.fits"


x_data = fits.getdata(x_path)
y_data = fits.getdata(y_path)
z_data = fits.getdata(z_path)
x_data_new = fits.getdata(x_path_new)


In [ ]:
sigma_kpc = 10 / 2.355  # target sigma in kpc
pixel_size_kpc = pixel_size_arcsec * length_per_arcsec
sigma_pix = np.ceil((sigma_kpc * (u.kpc) / pixel_size_kpc).value) # ≈ 2.54

In [ ]:
plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams.update({
    "font.size": 12,
    "font.family": "serif",
    "axes.labelsize": 13,
    "axes.titlesize": 13,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 11,
    "pdf.fonttype": 42  # Ensures TrueType font embedded (for APJ)
})



def save_figure(fig, filename, dpi=300):
    """Save figure in both PDF (for APJ) and PNG (for local use) formats."""
    base = os.path.join("figures", filename)
    fig.savefig(f"{base}.pdf", bbox_inches='tight')  # APJ-compatible vector format
    #fig.savefig(f"{base}.png", dpi=dpi, bbox_inches='tight')  # For checking / internal use
    print(f"Saved: {base}.pdf and {base}.png")


def crop_center(data, center, radius_pix):
    x0, y0 = int(center[1]), int(center[0])
    return data[x0 - radius_pix : x0 + radius_pix, y0 - radius_pix : y0 + radius_pix]

image_shape = x_data.shape
center = (image_shape[0] // 2, image_shape[1] // 2)
radius_pix = int(r_pix)

x_crop = crop_center(x_data, center, radius_pix)
y_crop = crop_center(y_data, center, radius_pix)
z_crop = crop_center(z_data, center, radius_pix)
x_crop_new = crop_center(x_data_new, center, radius_pix)

x_smooth = gaussian_filter(x_crop, sigma=sigma_pix)
y_smooth = gaussian_filter(y_crop, sigma=sigma_pix)
z_smooth = gaussian_filter(z_crop, sigma=sigma_pix)
x_smooth_new = gaussian_filter(x_crop_new, sigma=sigma_pix)

In [ ]:
fig = plt.figure(figsize=(12, 5))  
gs = fig.add_gridspec(1, 3, wspace=0.05)
axes = [fig.add_subplot(gs[0, i]) for i in range(3)]


x_norm = x_smooth/(2e+6*pixel_size_arcsec**2)
y_norm = y_smooth/(2e+6*pixel_size_arcsec**2)
z_norm = z_smooth/(2e+6*pixel_size_arcsec**2)

stack = np.concatenate([x_norm.ravel(), y_norm.ravel(), z_norm.ravel()])
vmin = np.nanpercentile(stack, 0.5)
vmax = np.nanpercentile(stack, 99.5)
vmin = max(vmin, 1e-20)  

norm = LogNorm(vmin=vmin, vmax=vmax)

ims = []
for ax, data, title in zip(axes,
                           [x_norm, y_norm, z_norm],
                           ['X Projection', 'Y Projection', 'Z Projection']):
    im = ax.imshow(np.clip(data, vmin, None), norm=norm, origin='lower', cmap='afmhot')
    ims.append(im)
    ax.set_title(title)
    ax.set_xticks([]); ax.set_yticks([])

for ax in axes:
    ax.add_patch(Rectangle((50, 50), length_pix, 20, color='white'))
    ax.text(50, 75, f'{length_kpc} kpc', color='white', fontsize=15, fontname='serif')

for ax, img in zip(axes, [x_crop, y_crop, z_crop]):
    h, w = img.shape  # height, width of the cropped image
    ax.plot(w // 2, h // 2, 'ko', markersize=5)

# set color bar
cbar = fig.colorbar(ims[0], ax=axes, location='right', fraction=0.046, pad=0.04)
cbar.set_label(r'X-ray Flux [counts cm$^{-2}$ s$^{-1}$ arcsec$^{-2}$]',
               fontsize=14, labelpad=10)


# Modified to include more ticks
#ticks = np.logspace(np.log10(vmin), np.log10(vmax), num=4)  # Creates 10 evenly spaced ticks in log space
ticks = np.array([1e-9, 1e-8,1e-7])
tick_labels = [r'$10^{-9}$', r'$10^{-8}$', r'$10^{-7}$']
cbar.set_ticks(ticks)
cbar.set_ticklabels(tick_labels)

cbar.outline.set_linewidth(1.2)

plt.tight_layout()
save_figure(plt.gcf(), "halo_0_xyz_xray")
plt.show()